### 11-7. 프로젝트: 멋진 작사가 만들기

##### Step 1. 데이터 다운로드
___
먼저 아래 링크에서 Song Lyrics 데이터를 다운로드해 주세요! 저장된 파일을 압축 해제한 후, 모든 txt 파일을 lyrics 폴더를 만들어 그 속에 저장해주세요!

* [Song Lyrics](https://www.kaggle.com/paultimothymooney/poetry/data)  
아래의 명령어를 실행하셔도 됩니다.

```
wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기
```

##### Step 2. 데이터 읽어오기
___
glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['The first words that come out', 'And I can see this song will be about you', "I can't believe that I can breathe without you"]


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
print(sentence)

Keep your hands high what


##### Step 3. 데이터 정제
___
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

**preprocess_sentence()** 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 문장을 **토큰화 했을 때 토큰의 개수가 15개**를 넘어가면 잘라내기를 권합니다.

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) <= 5: continue
    corpus.append(preprocess_sentence(sentence))

In [5]:
print(corpus[0])

<start> the first words that come out <end>


In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000, filters=' ', oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   6 241 ...   0   0   0]
 [  2   8   5 ...   0   0   0]
 [  2   5  32 ...   0   0   0]
 ...
 [  2 240   1 ...   0   0   0]
 [  2  10 502 ...   0   0   0]
 [  2 129  20 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f8efdbd0290>


In [7]:
tensors = []

for i in range(len(tensor)):
    if tensor[i][1] == tensor[i][2]: continue
    if tensor[i][2] == tensor[i][3]: continue
    tensors.append(tensor[i,:])

print(tensors[0])

[  2   6 241 424  17  67  56   3   0   0   0   0   0   0   0]


In [8]:
tensors = np.array(tensors)

In [9]:
type(tensors)

numpy.ndarray

In [10]:
x_train = tensors[:, :-1]
y_train = tensors[:, 1:]

print(x_train[0])
print(y_train[0])

[  2   6 241 424  17  67  56   3   0   0   0   0   0   0]
[  6 241 424  17  67  56   3   0   0   0   0   0   0   0]


##### Step 4. 평가 데이터셋 분리
훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!

```
enc_train, enc_val, dec_train, dec_val = <코드 작성>
```

여기까지 올바르게 진행했을 경우, 아래 실행 결과를 확인할 수 있습니다.

```
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
```

```
out:

Source Train: (124960, 14)
Target Train: (124960, 14)
```

만약 결과가 다르다면 천천히 과정을 다시 살펴 동일한 결과를 얻도록 하세요!

In [11]:
enc_train, enc_val, dec_train, dec_val = train_test_split(x_train, y_train, test_size=0.2)

In [12]:
enc_train = enc_train[:124960]
dec_train = dec_train[:124960]

In [13]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124960, 14)
Target Train: (124960, 14)


In [14]:
BUFFER_SIZE = len(x_train)
BATCH_SIZE = 256
steps_per_epoch = len(x_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

##### Step 5. 인공지능 만들기
___
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

```
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
```

데이터가 커서 훈련하는 데 시간이 제법 걸릴 겁니다. 여유를 가지고 작업하시면 좋아요 :)

```
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
```

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [19]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-7.7590157e-05,  5.9371734e-05, -9.9167257e-05, ...,
         -7.5823576e-05,  5.8274953e-05,  2.0059713e-04],
        [-3.5009021e-04, -6.6554476e-06, -7.3485455e-05, ...,
          8.1538076e-07,  1.3442989e-04,  2.4812450e-04],
        [-6.7999307e-04,  2.9468918e-04, -3.0101993e-04, ...,
          2.4471950e-04,  6.0984516e-05,  3.9856564e-04],
        ...,
        [ 8.6344208e-04,  9.6043007e-04, -8.0057612e-04, ...,
          5.2677497e-04, -6.0627202e-04, -3.9102859e-04],
        [ 1.0193330e-03,  9.4258948e-04, -7.5159653e-04, ...,
          7.4957887e-04, -6.2180770e-04, -5.8369484e-04],
        [ 9.3325455e-04,  8.6145604e-04, -8.0714579e-04, ...,
          7.1952899e-04, -5.1546452e-04, -8.6031161e-04]],

       [[-7.7590157e-05,  5.9371734e-05, -9.9167257e-05, ...,
         -7.5823576e-05,  5.8274953e-05,  2.0059713e-04],
        [-1.9743078e-05, -2.4274697e-04, -2.0811986e-04, ...,
         -2.1020265e-04, 

In [20]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [21]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, batch_size=32, epochs=10)

Epoch 1/10
3905/3905 [==============================] - 130s 33ms/step - loss: 3.3163
Epoch 2/10
3905/3905 [==============================] - 138s 35ms/step - loss: 2.8599
Epoch 3/10
3905/3905 [==============================] - 137s 35ms/step - loss: 2.5703
Epoch 4/10
3905/3905 [==============================] - 137s 35ms/step - loss: 2.3080
Epoch 5/10
3905/3905 [==============================] - 137s 35ms/step - loss: 2.0760
Epoch 6/10
3905/3905 [==============================] - 137s 35ms/step - loss: 1.8710
Epoch 7/10
3905/3905 [==============================] - 137s 35ms/step - loss: 1.6919
Epoch 8/10
3905/3905 [==============================] - 137s 35ms/step - loss: 1.5376
Epoch 9/10
3905/3905 [==============================] - 123s 31ms/step - loss: 1.4066
Epoch 10/10
3905/3905 [==============================] - 122s 31ms/step - loss: 1.2979


In [33]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [34]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love ma little nasty girl <end> '

#### 1. 실력 미숙으로 인하여, In [8], In [9] 사이에서 중복된 소절을 제거를 하고자 하였으나, 못함.
#### 2. 또한 loss값이 제시된 2.2보다 낮은 걸로 보아, 오버피팅이 일어난 듯 싶음.